In [1]:
import os
import re
import sys

In [2]:
cyrest_folder="/Users/jboucas/py2cytoscape/py2cytoscape/cyrest/"
docs_folder="/Users/jboucas/py2cytoscape/docs/cyrest/"

In [3]:
files=os.listdir(cyrest_folder)
no_files=["cyrest.py","__init__.py",'base.py']
files=[ s for s in files if s not in no_files ]
files=[ s for s in files if "pyc" not in s ]
files

['diffusion.py',
 'vizmap.py',
 'idmapper.py',
 'command.py',
 'edge.py',
 'cybrowser.py',
 'layout.py',
 'session.py',
 'view.py',
 'network.py',
 'group.py',
 'table.py',
 'node.py',
 'commands.py']

In [7]:
for f in ["table.py"]:
    print f
    sys.stdout.flush()
    functions={}
    mainspace=f.split(".")[0]
    print mainspace
    with open(cyrest_folder+f, "r") as fin:
        funcstring=None
        for l in fin:
            l_=l.rstrip("\n")
            if ("    def " in l_) & ("__" not in l_):
                fname=re.search('def (.*)\(', l_)
                fname=fname.group(1)
                func_title="## ***cyclient.%s.%s***" %(mainspace,fname)
                #print l_
                if "\\" not in l_:
                    func=re.search('def (.*):', l_)
                    func=func.group(1)
                    func="**`cyclient.%s.%s`**" %(mainspace,func)
                    functions[fname]={'title':func_title,'func':func}
                else:
                    funcstring=l_.strip("\\")
                    
            elif funcstring:
                funcstring=funcstring+l_
                if ":" in funcstring:
                    func=re.search('def (.*):', funcstring)
                    func=func.group(1)
                    
                    funcstring=None    
                    func="**`cyclient.%s.%s`**" %(mainspace,func)
                    functions[fname]={'title':func_title,'func':func}
        
    
    with open(cyrest_folder+f, "r") as fin:
        collect=False
        doc=[]
        function_found=False
        for l in fin:
            l_=l.rstrip("\n")
            if ("    def " in l_) & ("__" not in l_):
                fname=re.search('def (.*)\(', l_)
                fname=fname.group(1)
                func_title="## ***cyclient.%s.%s***" %(mainspace,fname) 
                function_found=True
            if ('"""' in l_ ) & (len( [s for s in doc if '"""' in s ]) == 0 ) & (function_found):
                collect=True
            if ('"""' in l_ ) & (len([s for s in doc if '"""' in s ]) == 1  ) & (function_found):
                functions[fname]["docs"]=doc
                doc=[]
                collect=False
            if (collect) & (function_found):
                doc.append( l_.rstrip("\n") )
    
    with open(docs_folder+f.replace(".py", ".md"), "w") as fout:
                     
        for docf in functions.keys():

            doc_=functions[docf]["docs"][1:]

            collect=True
            description=[]
            for s in doc_:
                if (":param" not in s) & (collect==True):
                    desc=s.strip(" ")
                    if len(desc) > 0:
                        description.append(desc)
                elif ":param" in s:
                    collect=False
            
            description="\n".join(description)

            params=[]
            params_={}
            pname=None
            for p in doc_:
                if ":param " in p:
                    pname=re.search(':param (.*):', p)
                    pname=pname.group(1)
                    params.append(pname)
                    pdesc=p.split("%s:"%pname)[-1].strip(" ")
                    params_[pname]=pdesc
                elif pname:
                    pdesc=params_[pname]
                    pdesc=pdesc+"\n"+p.strip(" ")
                    params_[pname]=pdesc
            params_text=[]
            for p in params:
                text="* **`%s`** %s" %(p, params_[p])
                params_text.append(text)


            ftitle=functions[docf]["title"]
            func=functions[docf]["func"]
            docs_text=ftitle+"\n\n"+func+"\n\n"+description+"\n\n"+"\n".join(params_text)+"\n\n___\n\n"
            docs_text=docs_text.replace(":returns:", "* **`returns`**")
            
            print docs_text
        
            fout.write(docs_text)

table.py
table
